In [1]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

In [2]:
from peft import PeftModel, PeftConfig

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

peft_model = PeftModel.from_pretrained(peft_model_base, 
                                       '/Users/nash/Project/peft/peft-dialogue-summary-checkpoint-local/', 
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)

In [11]:
print(peft_model.print_trainable_parameters())

trainable params: 0 || all params: 251116800 || trainable%: 0.0
None


In [12]:
from fedn.utils.helpers.helpers import get_helper, save_metadata, save_metrics

In [13]:
HELPER_MODULE = 'numpyhelper'
helper = get_helper(HELPER_MODULE)

In [17]:
def save_parameters(model, out_path):
    """ Save model paramters to file.

    :param model: The model to serialize.
    :type model: torch.nn.Module
    :param out_path: The path to save to.
    :type out_path: str
    """
    #np.set_bfloat16_enabled(True) 
    np.
    parameters_np = [val.float().cpu().numpy() for _, val in model.state_dict().items()]
    helper.save(parameters_np, out_path)

In [18]:
def init_seed(out_path='./compute_package/seed.npz'):
    """ Initialize seed model and save it to file.

    :param out_path: The path to save the seed model to.
    :type out_path: str
    """
    # Init and save
    model = peft_model
    save_parameters(model, out_path)

In [19]:
init_seed()

In [20]:
def load_parameters(model_path):
    """ Load model parameters from file and populate model.

    param model_path: The path to load from.
    :type model_path: str
    :return: The loaded model.
    :rtype: torch.nn.Module
    """
    model = compile_model()
    parameters_np = helper.load(model_path)

    params_dict = zip(model.state_dict().keys(), parameters_np)
    state_dict = collections.OrderedDict({key: torch.tensor(x) for key, x in params_dict})
    model.load_state_dict(state_dict, strict=True)
    return model